In [1]:
import pandas as pd

In [5]:
df = pd.read_csv("diffusion_db_unaltered.csv")

In [7]:
df

,prompt,image_nsfw,prompt_nsfw
0,"a portrait of a female robot made from code, v...",0.554853,0.001621
1,a portrait of a female robot made from a cloud...,0.153645,0.000707
2,"only memories remain, trending on artstation",0.062496,0.000425
3,dream swimming pool with nobody,0.030799,0.003586
4,a dog doing weights. epic oil painting.,0.181035,0.030822
...,...,...,...
1999995,david bowie giving a piggy back ride to ziggy ...,0.152736,0.022676
1999996,david bowie giving a piggy back ride to ziggy ...,0.127884,0.014115
1999997,funny computer,0.093476,0.000463
1999998,hilarious witty computing machine,0.124379,0.002547


In [145]:
df['nsfw'] = (df['image_nsfw'] > 0.7) | (df['prompt_nsfw'] > 0.7)
df

,prompt,image_nsfw,prompt_nsfw,nsfw
0,"a portrait of a female robot made from code, v...",0.554853,0.001621,False
1,a portrait of a female robot made from a cloud...,0.153645,0.000707,False
2,"only memories remain, trending on artstation",0.062496,0.000425,False
3,dream swimming pool with nobody,0.030799,0.003586,False
4,a dog doing weights. epic oil painting.,0.181035,0.030822,False
...,...,...,...,...
1999995,david bowie giving a piggy back ride to ziggy ...,0.152736,0.022676,False
1999996,david bowie giving a piggy back ride to ziggy ...,0.127884,0.014115,False
1999997,funny computer,0.093476,0.000463,False
1999998,hilarious witty computing machine,0.124379,0.002547,False


In [147]:
df['nsfw'].value_counts()[True]

117442

In [149]:
df['nsfw'].value_counts()[False]

1882558